## **Introduction to Data Science - Nhập môn khoa học dữ liệu - CSC14119**
### **HCMUS - Trường Đại học khoa học tự nhiên - Nov 2024.**
### **Đồ án thực hành cuối kì - Handling Real-World Problem.**
#### **Due:** 24/12/2024.
#### **Lớp:** 22_21.
#### **Giảng viên hướng dẫn:** Thầy Lê Ngọc Thành - Thầy Lê Nhựt Nam.
#### **STT nhóm:** 9.
---
### **Data Modeling - 03**
**Problem**: Trong cuộc sống hiện đại, nhiều người thường gặp khó khăn khi quyết định nấu món ăn nào dựa trên các nguyên liệu có sẵn trong nhà bếp. Điều này không chỉ dẫn đến sự lãng phí thực phẩm mà còn khiến quá trình nấu nướng mất thời gian hơn. Vì vậy, cần một giải pháp để giúp người dùng nhanh chóng tìm kiếm và đề xuất các món ăn phù hợp dựa trên danh sách nguyên liệu có sẵn.

**Description**: Mô hình gợi ý món ăn dựa trên nguyên liệu là một hệ thống đề xuất (recommendation system) sử dụng dữ liệu về nguyên liệu và công thức nấu ăn để cung cấp các gợi ý phù hợp cho người dùng. Người dùng sẽ nhập danh sách nguyên liệu hiện có, và mô hình sẽ phân tích dữ liệu để đưa ra các món ăn khả thi, sắp xếp theo mức độ phù hợp. 

**Solution**: Sử dụng mô hình K-Nearest Neighbors để đề xuất.

### 1. Thuật toán KNN

In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

In [2]:
ingredients_df = pd.read_csv("../Assert/ingredients.csv",sep=",")
ingredients_df.drop_duplicates(keep='first',inplace=True)
ingredients_df.head()

,Name of dish,soda,turmeric mixture,herb,yogurt,butter,beef bone,chili lemongrass fish sauce,beef fillet,chipotle smoked pepper powder,...,kiwi,lime juice,tea leaves,white sesame,eel,green onion,ice cubes,pigeon,chinese sausages,white vinegar
0,10 common problems and mistakes when making bread,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11 ways to use leftover egg yolks,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12 types of nuts for baking,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,14 ways to make Sponge cake/Gato for birthday ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,14 ways to use leftover egg whites,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# Tách nguyên liệu (input) và tên món ăn (label)
ingredients = ingredients_df.iloc[:,1:].astype(bool)
dishes = ingredients_df.iloc[:,0]

# Khởi tạo và huấn luyện mô hình KNN, do dữ liệu đầu vào là mảng chứa {0,1} (chỉ quan tâm có xuất hiện hay không)
# nên ta sẽ sử dụng Jaccard Distance để tính khoảng cách thay vì Euclidean Distance
k = 5
knn = KNeighborsClassifier(n_neighbors=k, metric='jaccard')
knn.fit(ingredients, dishes)

KNeighborsClassifier(metric='jaccard')

In [4]:
def dishesRecommenderKNN(ingredients_input: pd.DataFrame):
    distances, indices = knn.kneighbors(ingredients_input)
    filtered_dishes = []

    for i, dist in enumerate(distances[0]):
        if dist < 1:  # Chỉ chọn các món có khoảng cách nhỏ hơn 1
            filtered_dishes.append(dishes.iloc[indices[0][i]])

    return filtered_dishes

In [5]:
def convertInputToVector(input):
    ingredients_list = ingredients_df.columns[1:]
    input_df = pd.DataFrame([ingredients_list.isin(input)],columns=ingredients_list)

    return input_df

In [6]:
# Hàm này lấy ra danh sách nguyên liệu từ các món ăn cho trước
def cookingRecipes(predict):
    recipes_map = {}
    recipes_df = ingredients_df[ingredients_df["Name of dish"].isin(predict)]
    for _, row in recipes_df.iterrows():
        ingredients_list = list(row[row == 1].index)
        recipes_map[row["Name of dish"]] = ingredients_list

    return recipes_map

In [7]:
# Hàm in nguyên liệu của từng món ăn
def printRecipes(recipes_map):
    for key, value in recipes_map.items():
        print(f'Recipe: {key}')
        print(f'Ingredients: ')
        for i in value:
            print(f'- {i}')

In [8]:
input = ['shrimp']
input_df = convertInputToVector(input)

predict = dishesRecommenderKNN(input_df)
for i, recipe in enumerate(predict):
    print(f"{i + 1}. {recipe}")

1. Shrimp Floss Using Cooking Robot Team Cuisine
2. How to Make Crispy Fried Shrimp and Pork Dumplings Using an Air Fryer
3. Grilled Shrimp Skewers
4. Coconut Fried Shrimp With Cosori Air Fryer
5. Radish Soup Rolled with Shrimp and Pork


### 2. Giao diện GUI

In [9]:
import tkinter as tk
from tkinter import messagebox

In [10]:
class RecipeRecommender:
    def __init__(self, root):
        self.root = root
        self.root.title("Hệ thống gợi ý món ăn")

        # Kích thước cửa sổ
        self.window_width = 800
        self.window_height = 500

        # Lấy kích thước màn hình
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()

        # Tính toán tọa độ x và y để cửa sổ ở chính giữa
        x = (screen_width // 2) - (self.window_width // 2)
        y = (screen_height // 2) - (self.window_height // 2)

        # Đặt kích thước và vị trí cửa sổ
        self.root.geometry(f"{self.window_width}x{self.window_height}+{x}+{y}")

        self.bg_photo = tk.PhotoImage(file="Background.png")

        # Canvas
        self.canvas = tk.Canvas(self.root, width=self.window_width, height=self.window_height, highlightthickness=0)
        self.canvas.grid(row=0, column=0, sticky="nsew")

        # Đặt ảnh nền
        self.canvas.create_image(0, 0, anchor="nw", image=self.bg_photo)

        # Thiết lập grid để Canvas mở rộng khi thay đổi kích thước
        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_columnconfigure(0, weight=1)

        # Label
        self.label = tk.Label(self.canvas,text="Input ingredients:",bg="brown",fg='white',font=("calibri",16,"bold"),relief='solid')
        self.label.place(x=145, y=50)

        # Entry để nhập nguyên liệu
        self.entry = tk.Entry(self.canvas,width=27,font=("Calibri", 16),relief='solid',bd=2)
        self.entry.place(x=80, y=100)

        # Button để gợi ý món ăn
        self.recommend_button = tk.Button(self.canvas,text="ENTER",command=self.recommend,bg='brown',fg='white',font=("calibri",16,"bold"),relief=tk.RAISED)
        self.recommend_button.place(x=190, y=150)

        # Label để hiển thị kết quả
        self.result_label = tk.Label(self.canvas,text="",justify="left",anchor="w",font=("calibri",16),width=58,height=6,relief="solid")
        self.result_label.place(x=70, y=240) 

    def recommend(self):
        ingredients = self.entry.get()
        
        if not ingredients.strip():
            messagebox.showwarning("Cảnh báo", "Hãy nhập nguyên liệu!")
            self.result_label.config(text="")
            return
        
        ingredients_list = [ingredient.strip() for ingredient in ingredients.split(",")]
        processed_ingredients = convertInputToVector(ingredients_list)
        result = dishesRecommenderKNN(processed_ingredients)

        if result:
            recipe = ""
            for i, dish in enumerate(result):
                recipe += f"{i + 1}. {dish}\n"
            self.result_label.config(text=recipe)
        else:
            self.result_label.config(text="Không tìm thấy món ăn phù hợp!")

In [11]:
root = tk.Tk()
app = RecipeRecommender(root)
root.mainloop()